In [23]:
# Import pandas
import pandas as pd
import numpy as np

# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from pathlib import Path

import tensorflow as tf

# Import Gradio
import gradio as gr

In [24]:
# Set the model's file path
model_path = Path("milking_the_data.keras")

# Load the model to a new object
model = tf.keras.models.load_model(model_path)
model

<Sequential name=sequential, built=True>

In [25]:
import pickle

# Load the scaler from the file
with open('X_scaler.pkl', 'rb') as f:
    X_scaler = pickle.load(f)

In [26]:
# Define the mapping for months
month_mapping = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

In [27]:
def predict_milk_production(lact_num, avg_thi3, avg_csi3, avg_csi30, avg_precspring,
                            date_diff, calv_month, calv_day_of_month, alp_month, alp_day_of_month):

    # Convert month names to numerical values
    calv_month = month_mapping[calv_month]
    alp_month = month_mapping[alp_month]
    
    # Prepare the input data for the model
    input_data = np.array([[lact_num, avg_thi3, avg_csi3, avg_csi30, avg_precspring,
                            date_diff, calv_month, calv_day_of_month, alp_month, alp_day_of_month]])
    #scale the data
    input_data_scaled = X_scaler.transform(input_data)
    # Make prediction
    prediction = model.predict(input_data_scaled)
    
    return prediction[0][0]  # Return the predicted value


In [28]:
# Define the Gradio interface
inputs = [
    gr.Number(label='Lactation Number (number of births)'),
    gr.Slider(minimum=0, maximum=100, step=1, label='Average Temp Humidity Index (3 days)'),
    gr.Slider(minimum=700, maximum=1300, step=10, label='Average Cold Stress Index (3 days)'),
    gr.Slider(minimum=700, maximum=1300, step=10, label='Average Cold Stress Index (30 days)'),
    gr.Slider(minimum=0, maximum=500, step=10, label='Average Precipitation Spring (mm/month)'),
    gr.Slider(minimum=1, maximum=365, step=1, label='Number of Days Between Calving and Arriving in the Alp'),
    gr.Dropdown(choices=["January", "February", "March", "April", "May", "June", "July", 
                         "August", "September", "October", "November", "December"], label='Calving Month'),
    gr.Dropdown(choices=list(range(1, 32)), label='Day Calving Month'),  # Dropdown for 1-31
    gr.Dropdown(choices=["January", "February", "March", "April", "May", "June", "July", "August",
                         "September", "October", "November", "December"], label='Alping Month'),
    gr.Dropdown(choices=list(range(1, 32)), label='Day of Alping Month')  # Dropdown for 1-31
]

outputs = gr.Textbox(label='Predicted Milk Production')

interface = gr.Interface(fn=predict_milk_production, inputs=inputs, outputs=outputs)

# Launch the interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
